# Chapter 14: Recurrent Neural Networks

## Exercises

### 1. Can you think of a few applications for a sequence-to-sequence RNN? What about a sequence-to-vector RNN? And a vector-to-sequence RNN?

A sequence-to-sequence RNN is generally used for a model for predicting the future behavior of some input sequence. This can be used to create a predictive model for determining what the next word you are about to type might be.

A sequence-to-vector RNN is good for classifying sequences, such as sentiment analysis. Also a sequence-to-vector RNN is used for finding the embeddings of a vocabulary of words in a denser, smaller vector space.

In machine translation, vector-to-sequence RNN is good for decoding embeddings from input in one language into words in the target language. You can also use vector-to-sequence RNNs to generate captions for images.

### 2. Why do people use encoder-decoder RNNs rather than plain sequence-to-sequence RNNs for automatic translation?

Most models for automatic translation encode vocabularies as a vector space where each word is a perpendicular unit vector. For a vocabulary of 50,000 words, this means the input sequences are vectors in a 50,000-dimensional space. Training a sequence-to-sequence RNN for machine translation with large vocabularies would take a large amount of memory, making it inefficient.

Using an Encoder-Decoder model allows you to train the encoder to find a denser representation of the words, making training more efficient. Also training the model to find an embedding also helps the model learn what words are closely related to one another.

### 3. How could you combine a convolutional neural network and an RNN to classify videos?

Since a video is a sequence of images, you could create a convolutional neural network where each cell is a convolutional layers which learn feature maps for the images in each frame of the video. You could have it learn one set of feature maps for the input and another feature map for the previous output.

### 4. What are the advantages of building an RNN using `dynamic_rnn()` rather than `static_rnn()`?

The `static_rnn()` function creates new graph nodes for each time step in the sequence. This means that if you are processing a sequence with a large number of steps, you risk getting an OOM error when building your TensorFlow graph. The `dynamic_rnn()` function uses a while loop to perform multiple operations using the same nodes. The `dyanmic_rnn()` function also allows you to swap memory between the GPU and the CPU using the `swap_memory` parameter. It also accepts a single tensor as an input instead of a list of tensors for each time step in the sequence.

### 5. How can you deal with variable-length input sequences? What about variable-length output sequences?

The `dynamic_rnn()` function takes a `sequence_length` parameter which is a 1D tensor of integers which represent the sequence length of each of the inputs. Input sequences that are less than the maximum length sequence are padded with zeros.

For variable-length output sequences, since it is not possible to determine how long each output will be prior to training, so each output sequence ends with an end of sequence (EOS) character to delimit the end.

### 6. What is a common way to distribute training and execution of a deep RNN across multiple GPUs?

In order to distribute an RNN across devices you cannot just simply call the `tf.device()` function. This is because TensorFlow's built in RNN cell classes like `BasicRNNCell` do not create the graph ncdes themselves, rather they are cell factories.

In order to distribute an RNN across devices, you must define a new cell factory which actually creates each cell on a separate device. For an example, see `DeviceCellWrapper` in `RecurrentNeuralNetworks.ipynb`.

### 7. _Embedded Reber grammars_ are artificial grammars used to produce strings. Train an RNN to identify whether or not a string represents the grammar discussed in [Jenny Orr's introduction](http://www.willamette.edu/~gorr/classes/cs449/reber.html) or not. You will first need to write a function capable of generating a training batch containing about 50% strings that respect the grammar and 50% that do not.

In [0]:
# Define a function to generate Rebber grammars. Each key in the dict
# is a node in the graph. Each element in the list is an adjacent node
# in the graph.

import numpy as np

# Adjacency list for the graph.
reber_grammar_graph = [
  [('B', 1)],
  [('T', 2), ('P', 3)],
  [('S', 2), ('X', 4)],
  [('T', 3), ('V', 5)],
  [('X', 3), ('S', 6)],
  [('P', 4), ('V', 6)],
  [('E', None)],
]

def generate_reber_grammar():
  idx = 0
  result = ''
  while idx is not None:
    chars = reber_grammar_graph[idx]
    c, idx = chars[np.random.randint(0, len(chars))]
    result += c
  return result

In [2]:
generate_reber_grammar()

'BTSXXTTTTTTTTTTTVVE'

In [0]:
# Defining a function for generating embedded Reber grammar.

REBER_GRAPH = 'reber_graph'

embedded_reber_grammar_graph = [
  [('B', 1)],
  [('T', 2), ('P', 3)],
  [(REBER_GRAPH, 4)],
  [(REBER_GRAPH, 5)],
  [('T', 6)],
  [('P', 6)],
  [('E', None)]
]

def generate_embedded_reber_grammar():
  idx = 0
  result = ''
  while idx is not None:
    chars = embedded_reber_grammar_graph[idx]
    c, idx = chars[np.random.randint(0, len(chars))]
    result += c if c != REBER_GRAPH else generate_reber_grammar()
  return result

In [36]:
generate_embedded_reber_grammar()

'BPBPVPXVVEPE'

In [0]:
# Generate a corrupted string by creating an embedded Reber grammar
# and then change a single character.

def generate_corrupted_string():
  erg_string = generate_embedded_reber_grammar()
  chars = set(erg_string)
  idx = np.random.randint(0, len(erg_string))
  bad_char = np.random.choice(list(chars - set(erg_string[idx])))
  return '{}{}{}'.format(erg_string[:idx], bad_char, erg_string[idx+1:])

In [38]:
generate_corrupted_string()

'BPBPPVVEPE'

In [0]:
# One-hot encoding each string.

char_to_idx_map = {c: i for i, c in enumerate('BEPSTVX')}

def char_to_vector(c):
  data = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  data[char_to_idx_map[c]] = 1.0
  return data

def one_hot_encode(erg_str):
  return np.array([char_to_vector(c) for c in erg_str], dtype=np.float32)

In [41]:
one_hot_encode(generate_embedded_reber_grammar())

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [0]:
# Pad a one-hot encoded embedded Reber grammar string.
def pad_zeros(ohe_erg_str, length):
  str_length = len(ohe_erg_str)
  if str_length > length:
    raise Exception(
        'the 2nd argument of pad_zeros must be gte the length of the first '
        'argument')
  for i in range(length - str_length):
    ohe_erg_str = \
        np.concatenate((ohe_erg_str, [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]))
  return ohe_erg_str

In [52]:
pad_zeros(one_hot_encode(generate_embedded_reber_grammar())[:10], 15)

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [0]:
# Generate a single training batch.

def generate_batch(batch_size, max_seq_length):
  good_batch = []
  bad_batch = []
  while len(good_batch) < batch_size / 2:
    seq = one_hot_encode(generate_embedded_reber_grammar())
    if len(seq) > max_seq_length:
      continue
    good_batch.append((seq, True))
  while len(bad_batch) < batch_size / 2:
    seq = one_hot_encode(generate_corrupted_string())
    if len(seq) > max_seq_length:
      continue
    bad_batch.append((seq, False))
  batch = []
  seq_lengths = []
  labels = []
  all_seqs = good_batch + bad_batch
  np.random.shuffle(all_seqs)
  for seq, is_valid in all_seqs:
    batch.append(pad_zeros(seq, max_seq_length))
    seq_lengths.append(len(seq))
    labels.append([int(is_valid)])
  return np.array(batch, dtype=np.float32), \
      np.array(labels, dtype=np.int32), \
      np.array(seq_lengths, dtype=np.int32)

In [0]:
# Write the TensorFlow graph

import tensorflow as tf

n_chars = 7
max_sequence_length = 20
n_outputs = 1
n_neurons = 100
learning_rate = 0.01

graph = tf.Graph()

with graph.as_default():
  X = tf.placeholder(tf.float32, (None, max_sequence_length, n_chars))
  y = tf.placeholder(tf.float32, (None, 1))
  seq_length = tf.placeholder(tf.int32, (None))

  cell = tf.contrib.rnn.GRUCell(num_units=n_neurons)
  _, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32,
                                sequence_length=seq_length)
  logits = tf.layers.dense(states, n_outputs)

  xentropy = \
      tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(y, tf.float32),
                                              logits=logits)
  loss = tf.reduce_mean(xentropy)
  opt = tf.train.AdamOptimizer(learning_rate)
  training_op = opt.minimize(loss)

  y_pred = tf.cast(tf.greater(logits, 0.0), tf.float32)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(y, y_pred), tf.float32))

  init = tf.global_variables_initializer()

In [178]:
# Training a model to recognize embedded Reber grammars.

n_epochs = 50
n_batches = 25
batch_size = 100
validation_set_size = 200

with graph.as_default():
  with tf.Session() as sess:
    sess.run(init)
    X_valid, y_valid, seq_len_valid = \
        generate_batch(validation_set_size, max_sequence_length)
    valid_feed_dict = {
        X: X_valid,
        y: y_valid,
        seq_length: seq_len_valid,
    }
    for epoch in range(n_epochs):
      for _ in range(n_batches):
        X_batch, y_batch, seq_len_batch = \
            generate_batch(batch_size, max_sequence_length)
        sess.run(training_op, feed_dict={
            X: X_batch,
            y: y_batch,
            seq_length: seq_len_batch,
        })
      if epoch % 5 == 0:
        loss_val = loss.eval(feed_dict=valid_feed_dict)
        acc_val = accuracy.eval(feed_dict=valid_feed_dict)
        print('Epoch: {}\tLoss: {}\tAccuracy: {}'.format(epoch, loss_val,
                                                         acc_val))

Epoch: 0	Loss: 0.6584188938140869	Accuracy: 0.6449999809265137
Epoch: 5	Loss: 0.6347793340682983	Accuracy: 0.4300000071525574
Epoch: 10	Loss: 0.13499197363853455	Accuracy: 0.9599999785423279
Epoch: 15	Loss: 0.08909790217876434	Accuracy: 0.9750000238418579
Epoch: 20	Loss: 0.04386292025446892	Accuracy: 0.9900000095367432
Epoch: 25	Loss: 0.0029373581055551767	Accuracy: 1.0
Epoch: 30	Loss: 0.001209043781273067	Accuracy: 1.0
Epoch: 35	Loss: 0.0013871783157810569	Accuracy: 1.0
Epoch: 40	Loss: 0.00021878271945752203	Accuracy: 1.0
Epoch: 45	Loss: 0.0001390562829328701	Accuracy: 1.0


### 8. Tacle the ["How much did it rain? II" Kaggle competition](https://www.kaggle.com/c/how-much-did-it-rain-ii), this is a time series prediction task.

[Luis Andre Dutra e Silva's interview](http://blog.kaggle.com/2015/12/17/how-much-did-it-rain-ii-2nd-place-luis-andre-dutra-e-silva/) shows some insights that he used to reach second place in the competition.

In [0]:
# TODO